In [1]:
import os
import math
from dotenv import load_dotenv
from aocd import get_data

In [5]:
load_dotenv()
secret_key = os.getenv("SECRET_KEY")
os.environ['AOC_SESSION'] = secret_key

data = get_data(year=2023, day=7)

## Part 1
In Camel Cards, you get a list of hands, and your goal is to order them based on the strength of each hand. A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2. The relative strength of each card follows this order, where A is the highest and 2 is the lowest.

Every hand is exactly one type. From strongest to weakest, they are:
- Five of a kind, where all five cards have the same label: AAAAA
- Four of a kind, where four cards have the same label and one card has a different label: AA8AA
- Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
- Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
- Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
- One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
- High card, where all cards' labels are distinct: 23456
Hands are primarily ordered based on type; for example, every full house is stronger than any three of a kind.

If two hands have the same type, a second ordering rule takes effect. Start by comparing the first card in each hand. If these cards are different, the hand with the stronger first card is considered stronger. If the first card in each hand have the same label, however, then move on to considering the second card in each hand. If they differ, the hand with the higher second card wins; otherwise, continue with the third card in each hand, then the fourth, then the fifth.

So, 33332 and 2AAAA are both four of a kind hands, but 33332 is stronger because its first card is stronger. Similarly, 77888 and 77788 are both a full house, but 77888 is stronger because its third card is stronger (and both hands have the same first and second card).

To play Camel Cards, you are given a list of hands and their corresponding bid (your puzzle input). For example:
```
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
```
This example shows five hands; each hand is followed by its bid amount. Each hand wins an amount equal to its bid multiplied by its rank, where the weakest hand gets rank 1, the second-weakest hand gets rank 2, and so on up to the strongest hand. Because there are five hands in this example, the strongest hand will have rank 5 and its bid will be multiplied by 5.

So, the first step is to put the hands in order of strength:
- 32T3K is the only one pair and the other hands are all a stronger type, so it gets rank 1.
- KK677 and KTJJT are both two pair. Their first cards both have the same label, but the second card of KK677 is stronger (K vs T), so - KTJJT gets rank 2 and KK677 gets rank 3.
- T55J5 and QQQJA are both three of a kind. QQQJA has a stronger first card, so it gets rank 5 and T55J5 gets rank 4.
Now, you can determine the total winnings of this set of hands by adding up the result of multiplying each hand's bid with its rank (765 * 1 + 220 * 2 + 28 * 3 + 684 * 4 + 483 * 5). So the total winnings in this example are 6440.

Find the rank of every hand in your set. What are the total winnings?

In [6]:
# Card value mappings
card_dict = {c: i for i,c in enumerate('23456789TJQKA', start=2)}

# Hand type rankings
hand_types = {
    'five-of-a-kind': 7,
    'four-of-a-kind': 6,
    'full-house': 5,
    'three-of-a-kind': 4,
    'two-pair': 3,
    'one-pair': 2,
    'high-card': 1
}

def count_hand(hand):
    # Establish card frequencies
    card_counter = {c: 0 for c in '23456789TJQKA'}
    # Loop through each card
    for card in hand:
        # Update frequency dict
        card_counter[card] += 1
    # Get max frequency
    max_freq = max(card_counter.values())
    # Get distinct frequencies >0
    distinct_freqs = len(list(v for v in card_counter.values() if v > 0))
    
    return max_freq, distinct_freqs

def classify_hand(max_freq, distinct_freqs):
    # Determine hand type
    if max_freq == 5:
        hand_type = 'five-of-a-kind'
    elif max_freq == 4:
        hand_type = 'four-of-a-kind'
    elif max_freq == 3:
        if distinct_freqs == 2:
            hand_type = 'full-house'
        else:
            hand_type = 'three-of-a-kind'
    elif max_freq == 2:
        if distinct_freqs == 3:
            hand_type = 'two-pair'
        else:
            hand_type = 'one-pair'
    else:
        hand_type = 'high-card'
        
    return hand_type

def parse_input(data):
    hands = {}
    for i, hand_data in enumerate(data.splitlines()):
        # Split hand and bid
        hand, bid = hand_data.split()
        # Calculate hand type
        max_freq, distinct_freqs = count_hand(hand)
        hand_type = classify_hand(max_freq, distinct_freqs)
        # Append to dict
        hands[i] = {'hand': hand, 'bid': int(bid), 'hand_type': hand_type}
        
    return hands

def hand_key(i, hands_dict):
    hand_type, hand = hands_dict[i]['hand_type'], hands_dict[i]['hand']
    # First element: hand type value
    hand_type_value = hand_types[hand_type]
    # Second element: list of card values in original order
    card_values = [card_dict[c] for c in hand]
    
    return (hand_type_value, card_values)

def rank_hands(hands):
    # Rank hands
    ranked_hand_idx = sorted(hands.keys(), key=lambda x: hand_key(x, hands))
    for i,idx in enumerate(ranked_hand_idx):
        hands[idx]['rank'] = i+1
        
    return hands

def solve_part1(data):
    # Parse data
    hands = parse_input(data)
    # Rank hands
    hands = rank_hands(hands)
    # Calculate winnings
    winnings = sum([hands[i]['bid'] * hands[i]['rank'] for i in hands])

    return winnings

In [7]:
solve_part1(data)

246409899

## Part 2
To make things a little more interesting, the Elf introduces one additional rule. Now, J cards are jokers - wildcards that can act like whatever card would make the hand the strongest type possible.

To balance this, J cards are now the weakest individual cards, weaker even than 2. The other cards stay in the same order: A, K, Q, T, 9, 8, 7, 6, 5, 4, 3, 2, J.

J cards can pretend to be whatever card is best for the purpose of determining hand type; for example, QJJQ2 is now considered four of a kind. However, for the purpose of breaking ties between two hands of the same type, J is always treated as J, not the card it's pretending to be: JKKK2 is weaker than QQQQ2 because J is weaker than Q.

Now, the above example goes very differently:
```
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
```
32T3K is still the only one pair; it doesn't contain any jokers, so its strength doesn't increase.
KK677 is now the only two pair, making it the second-weakest hand.
T55J5, KTJJT, and QQQJA are now all four of a kind! T55J5 gets rank 3, QQQJA gets rank 4, and KTJJT gets rank 5.
With the new joker rule, the total winnings in this example are 5905.

Using the new joker rule, find the rank of every hand in your set. What are the new total winnings?

In [20]:
# Card value mappings
card_dict = {c: i for i,c in enumerate('J23456789TQKA', start=1)}

# Hand type rankings
hand_types = {
    'five-of-a-kind': 7,
    'four-of-a-kind': 6,
    'full-house': 5,
    'three-of-a-kind': 4,
    'two-pair': 3,
    'one-pair': 2,
    'high-card': 1
}

def count_hand(hand):
    # Establish card frequencies
    card_counter = {c: 0 for c in '23456789TJQKA'}
    # Loop through each card
    for card in hand:
        # Update frequency dict
        card_counter[card] += 1
    # Get max frequency
    max_freq = max(card_counter.values())
    # Get distinct frequencies >0
    distinct_freqs = len(list(v for v in card_counter.values() if v > 0))
    
    return card_counter, max_freq, distinct_freqs

def modify_hand(hand, card_counter):
    # If no jokers, no need to modify
    if 'J' not in hand:
        return hand
    
    # If all jokers, keep as is (will be five of a kind)
    if hand == 'JJJJJ':
        return hand
        
    # Find highest frequency non-joker card (if tied, take highest value)
    non_joker_cards = {k:v for k,v in card_counter.items() if k != 'J'}
    max_freq = max(non_joker_cards.values())
    max_cards = [k for k,v in non_joker_cards.items() if v == max_freq]
    best_card = max(max_cards, key=lambda x: card_dict[x])
    
    return hand.replace('J', best_card)

def classify_hand(max_freq, distinct_freqs):
    # Determine hand type
    if max_freq == 5:
        hand_type = 'five-of-a-kind'
    elif max_freq == 4:
        hand_type = 'four-of-a-kind'
    elif max_freq == 3:
        if distinct_freqs == 2:
            hand_type = 'full-house'
        else:
            hand_type = 'three-of-a-kind'
    elif max_freq == 2:
        if distinct_freqs == 3:
            hand_type = 'two-pair'
        else:
            hand_type = 'one-pair'
    else:
        hand_type = 'high-card'
        
    return hand_type

def parse_input(data):
    hands = {}
    for i, hand_data in enumerate(data.splitlines()):
        # Split hand and bid
        hand, bid = hand_data.split()
        # Modify hand
        card_counter, max_freq, distinct_freqs = count_hand(hand)
        modified_hand = modify_hand(hand, card_counter)
        # Calculate hand type
        card_counter, max_freq, distinct_freqs = count_hand(modified_hand)
        hand_type = classify_hand(max_freq, distinct_freqs)
        # Append to dict
        hands[i] = {'hand': hand, 'bid': int(bid), 'hand_type': hand_type, 'modified_hand': modified_hand}
        
    return hands

def hand_key(i, hands_dict):
    hand_type = hands_dict[i]['hand_type']
    orig_hand = hands_dict[i]['hand']  # Use original hand for card comparison
    
    # First element: hand type value
    hand_type_value = hand_types[hand_type]
    # Second element: list of card values in original order
    card_values = [card_dict[c] for c in orig_hand]
    
    return (hand_type_value, card_values)

def rank_hands(hands):
    # Rank hands
    ranked_hand_idx = sorted(hands.keys(), key=lambda x: hand_key(x, hands))
    for i,idx in enumerate(ranked_hand_idx):
        hands[idx]['rank'] = i+1
        
    return hands

def solve_part2(data):
    # Parse data
    hands = parse_input(data)
    # Rank hands
    hands = rank_hands(hands)
    # Calculate winnings
    winnings = sum([hands[i]['bid'] * hands[i]['rank'] for i in hands])

    return winnings

In [23]:
solve_part2(data)

244848487